In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

os.chdir('C:\wzl\实习\data')

feature_list = ['lat', 'lon',
                        'TMP_P0_L1_GLL0', 'SPFH_P0_2L108_GLL0', 'RH_P0_L4_GLL0',
                        'PWAT_P0_L200_GLL0', 'UGRD_P0_L6_GLL0', 'GUST_P0_L1_GLL0',
                        'PRES_P0_L7_GLL0', 'CultivatedLand', 'WoodLand', 'GrassLand', 'Waters',
                        'UrbanRural', 'UnusedLand', 'Ocean', 'ELEVATION', 'AOD', 'month',
                        'year', 'weekday']
label_name = 'o3'


In [2]:
from sklearn.preprocessing import MinMaxScaler
train = pd.read_csv('day\\train_set_daily_new.csv')
test = pd.read_csv('day\\test_set_daily_new.csv')

X_train, y_train, X_test, y_test = train[feature_list], train[label_name],test[feature_list],test[label_name]
X_train = pd.get_dummies(X_train,columns = ['month','year','weekday'],drop_first=True)
X_test = pd.get_dummies(X_test,columns = ['month','year','weekday'],drop_first=True)

In [3]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
class Mynet(nn.Module):
    def __init__(self,input_size,hidden1,hidden2,hidden3,hidden4,output_size,dropout_prob):
        super(Mynet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.fc2 = nn.Linear(hidden1,hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden2,hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        self.fc4 = nn.Linear(hidden3,hidden4)
        self.fc5 = nn.Linear(hidden4,output_size)
    def forward(self,x):
        x = self.fc1(x)
        feature1 = x
        x = self.bn1(x)
        x = F.relu(x)
    
        x = self.dropout(x)
        x = self.fc2(x)
        feature2 = x
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        feature3 = x
        x = self.bn3(x)
        x = F.relu(x)
        x = self.fc4(x)
        feature4 = x
        x = F.relu(x)
        out = self.fc5(x)
        return out,feature1,feature2,feature3,feature4

In [5]:
from torch.utils.data import TensorDataset,DataLoader
device = torch.device('cuda')
input = X_train.shape[1]
Mymodel = Mynet(input_size=input,hidden1 = 1680, hidden2 = 2560, hidden3 = 128, hidden4 = 64,output_size=1,
                dropout_prob=0.2).to(device)
X_train = torch.tensor(X_train, dtype = torch.float32, device = device)
X_test = torch.tensor(X_test, dtype = torch.float32, device = device)
y_train = torch.tensor(y_train, dtype = torch.float32, device = device)
y_test = torch.tensor(y_test, dtype = torch.float32, device = device)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)

train_data = DataLoader(train, batch_size = 4096, shuffle = True)


In [6]:
optimizer = torch.optim.Adam(Mymodel.parameters(),lr = 0.01)
Loss = nn.MSELoss()
Mymodel.train()
for i in range(300):
    loss_list= []
    for X,y in train_data:
        optimizer.zero_grad()
        out = Mymodel(X)[0]
        y = y.view(-1,1)
        loss = Loss(out,y)
        loss_list.append(loss)
        loss.backward()
        optimizer.step()
    print('episode',i,'loss',sum(loss_list)/len(loss_list))
    


episode 0 loss tensor(648.7390, device='cuda:0', grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [ ]:
#torch.save(Mymodel,'day\\NN5.pth')
Mymodel


Mynet(
  (fc1): Linear(in_features=37, out_features=1680, bias=True)
  (bn1): BatchNorm1d(1680, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1680, out_features=2560, bias=True)
  (bn2): BatchNorm1d(2560, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=2560, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
)

In [11]:
import torch
Mymodel = torch.load('day\\NN5.pth')

Mymodel.eval()
y_pred = Mymodel(X_test)[0].cpu().detach().numpy().flatten()
#y_test = y_test.cpu().detach().numpy()


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
smape = np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test))) * 100
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


from confidence import confidence
conf = confidence(y_test,y_pred,10)

print("RMSE:", rmse)
print("SMAPE:", smape)
print("MAE:", mae)
print("R2:", r2)
print('confidence',conf)

RMSE: 12.844414
SMAPE: 18.140986561775208
MAE: 9.57674
R2: 0.7947686167271085
confidence 0.6307052691832615
